In [29]:
import os
import re
import json
import pickle
import sqlite3
import jsonlines
from collections import Counter
from unicodedata import normalize
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
# import tensorflow as tf
# from tensorflow.keras import layers
# from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Flatten
# from tensorflow.keras.models import Model, load_model
# from tensorflow.keras.regularizers import l1, l2
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.utils import Sequence
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
# from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,ConfusionMatrixDisplay





# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# split big jsonl files to chunks

In [112]:
datasets = os.listdir(r"C:\Users\Saba\Desktop\Python\AI CS hackathon btu\datasets\websites\comprehensive_dataset")
def replace_unnecessary_symbols(text):
    filters = '1234567890!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r'
    translate_dict = {c: " " for c in filters}
    text = text.lower()
    # Create a translation table
    translation_table = str.maketrans(translate_dict)
    html_translate_dict = {"href":"",
                           "src": "",
                           " vc ":"",
                           "jpg":"",
                          " en ":"",
                          "thumb":"",
                          'title':"",
                          " com ": "",
                          " https ":"",
                          " ru ":"",
                          " net ": ""}
    # html_translation_table = str.maketrans(html_translate_dict)
    # Use translate method
    new_text = text.translate(translation_table)
    for key in html_translate_dict:
        new_text = new_text.replace(key,html_translate_dict[key])
    new_text = re.sub(r'\s+', ' ', new_text)
    return new_text
def save_json(filepath_to_save,json_index,dict):
    with open(f'{filepath_to_save}/{json_index}_html_cleaned.json', 'w') as f:
        json.dump(dict, f, indent=4)
def save_to_chunks(name,file):
    
    chunks = pd.read_json(file, lines=True, chunksize=1)
    num = 1
    for chunk in chunks:
        
        if num % 1000 == 0:
            print(filename)
        try:
            html_content = chunk["html"].values[0]
            soup  = BeautifulSoup(html_content, 'html')
        except:
            print(f"could not parse row number {num}")
            continue
        
        for tag in soup(['script', 'style', 'img',"input",'iframe']):
            tag.decompose()
        for a_tag in soup.find_all('a'):
            del a_tag['href']
        for tag in soup.find_all(True):  # True selects all tags
            if 'style' in tag.attrs:
                del tag.attrs['style'] 
            if 'class' in tag.attrs:
                del tag.attrs['class']
            if 'target' in tag.attrs:
                del tag.attrs['target']
                
        # we should get rid of tab and escape charaters, also weird symbols and replace duplicate spaces with one space
        html_text = soup.get_text(separator=' ')
        html_text = normalize('NFKD', html_text)
        
        html_text = replace_unnecessary_symbols(html_text)
        
        url = normalize('NFKD', chunk['url'].values[0])
        label = chunk['label'].values[0]
        
        processed_json = {
            "url":url,
            "label":label,
            "html":html_text
        }
        filepath_to_save = 'C:/Users/Saba/Desktop/Python/AI CS hackathon btu/datasets/websites/comprehensive_chunks_cleaned'
        save_json(filepath_to_save,num,processed_json)
        
        if num % 10 == 0:
            break
        num +=1
        
    




In [113]:
  
file = "C:/Users/Saba/Desktop/Python/AI CS hackathon btu/datasets/websites/comprehensive_dataset/"+datasets[0]
print(file)

C:/Users/Saba/Desktop/Python/AI CS hackathon btu/datasets/websites/comprehensive_dataset/adult_OWS_URL_HTML_DS.jsonl


In [114]:
save_to_chunks("adult",file)

# Load chunks of datasets

In [25]:
def load_chunk_to_dataframe(chunk_index):
    chunks_filepath = "C:/Users/Saba/Desktop/Python/AI CS hackathon btu/datasets/websites/comprehensive_chunks_parsed/"
    chunks = os.listdir(chunks_filepath)
    chunk_fullpath = chunks_filepath+chunks[chunk_index]
    df = pd.read_json(chunk_fullpath)
    # df.drop(columns=["sublabel",'crawled','uid'],inplace=True)
    return df



In [26]:
chunks_filepath = "C:/Users/Saba/Desktop/Python/AI CS hackathon btu/datasets/websites/comprehensive_chunks_parsed/"
chunks = os.listdir(chunks_filepath)
chunks_length = len(chunks)
# print(chunks_length)
json_files_full = [chunks_filepath+chunk for chunk in chunks]

In [27]:

def process_json_file(json_file):
    with open(json_file,'r') as file:
        json_data = json.load(file)
    key = list(json_data['html_text'].keys())[0]
    label = json_data["label"][key]
    url =  normalize('NFKD', json_data['url'][key])
    html_text = normalize('NFKD', json_data['html_text'][key])
    
    html_text = replace_unnecessary_symbols(html_text).replace("   "," ")
    processed_json = {
        "url":url,
        "label":label,
        "html":html_text
    }
    return processed_json


def process_json_files(json_files):
    jsons_length = len(json_files)
    filepath_to_save = r'C:\Users\Saba\Desktop\Python\AI CS hackathon btu\datasets\websites\comprehensive_chunks_cleaned'
    
    for json_index in range(jsons_length-1):
        json_file = json_files[json_index]
        
        extracted_text = process_json_file(json_file)
        
        with open(f'{filepath_to_save}/{json_index}_html_cleaned.json', 'w') as f:
            json.dump(extracted_text, f, indent=4) 
        
        if json_index % 2 == 0 and json_index != 0:
            print(json_index, end='\r')
            break



In [28]:
texts_list = process_json_files(json_files_full)
# print(len(texts_list))

In [ ]:
with open('htmls_parsed_text_list.pkl', 'wb') as f:
    pickle.dump(texts_list, f)
    print("list saved successfully")

In [6]:
with open(json_files_full[0],'r') as file:
    json_data = json.load(file)

{'label': {'0': 'Adult'},
 'url': {'0': 'https://www.imzog.com/en/all/latest/'},
 'html_text': {'0': 'Free Porn Sites ,Videos Porno Tube - IMzog.com Straight Gay Shemale All latest All popular Categories Our network See.xxx PornJ TheGay tPorn.xxx SSS.xxx DesiPorn.tube GetSex.xxx Porn555 PornForRelax PornGo.xxx More GermanySee.xxx tPorn.xxx GetSex.xxx PornJ PornQ Popular categories %href::/en/blowjob/||\r %title::Blowjob||\r %thumb:://imzog1.hclips.net/timages/zq/909410.jpg||\r %vc::719,731 %href::/en/big tits/||\r %title::Big Tits||\r %thumb:://imzog2.hclips.net/timages/wk/594459.jpg||\r %vc::582,563 %href::/en/panties/||\r %title::Panties||\r %thumb:://imzog2.hclips.net/timages/Jp/867869.jpg||\r %vc::39,138 %href::/en/pornstar/||\r %title::Pornstar||\r %thumb:://imzog1.hclips.net/timages/hba/3335500.jpg||\r %vc::634,970 %href::/en/russian/||\r %title::Russian||\r %thumb:://imzog1.hclips.net/timages/Bw/1223925.jpg||\r %vc::51,610 %href::/en/skinny/||\r %title::Skinny||\r %thumb:://imzo

In [13]:
translate_dict

{'!': ' ',
 '"': ' ',
 '#': ' ',
 '$': ' ',
 '%': ' ',
 '&': ' ',
 '(': ' ',
 ')': ' ',
 '*': ' ',
 '+': ' ',
 ',': ' ',
 '-': ' ',
 '.': ' ',
 '/': ' ',
 ':': ' ',
 ';': ' ',
 '<': ' ',
 '=': ' ',
 '>': ' ',
 '?': ' ',
 '@': ' ',
 '[': ' ',
 '\\': ' ',
 ']': ' ',
 '^': ' ',
 '_': ' ',
 '`': ' ',
 '{': ' ',
 '|': ' ',
 '}': ' ',
 '~': ' ',
 '\t': ' ',
 '\n': ' '}

In [ ]:
data

In [2]:
with open('C:/Users/Saba/Desktop/Python/AI CS hackathon btu/datasets/images/dickpics/german_mydickpictures_links.txt',"r") as f:
    lines = f.readlines()


In [5]:
print(len(lines[79544:]))

9396
